In [1]:
import pandas as pd
import numpy as np
import ast
import matplotlib.pyplot as plt
from matplotlib import font_manager, rcParams
import warnings, logging
import re

In [ ]:
ads_list = pd.read_csv("/Users/Jiwon/Documents/GitHub/final_project/Jiwon/광고도메인리스트.csv")
time_df = pd.read_csv("/Users/Jiwon/Documents/GitHub/final_project/Jiwon/수정_시간별적립보고서(최종).csv")
click = pd.read_csv("/Users/Jiwon/Documents/GitHub/final_project/Jiwon/유저테이블.csv")
cv_goal = pd.read_csv("/Users/Jiwon/Documents/GitHub/final_project/Jiwon/cv_goal.csv")

# ads

## ads_portfolio

In [ ]:
# epire 컬럼 만들기

# 1. 기준일 설정
today = pd.to_datetime('today').date()

# 2. expire 컬럼 초기화 (기본값 0)
ads_list['expire'] = 0

# 3. 실제 날짜 형태 필터링 및 만료 체크
def check_expire(date_str):
    try:
        # 9999나 0000이 포함된 경우는 무기한으로 처리 (expire = 0)
        if '9999' in str(date_str) or '0000' in str(date_str):
            return 0
        
        # 실제 날짜로 변환 시도
        end_date = pd.to_datetime(date_str).date()
        
        # 오늘보다 이전이면 만료 (1), 이후면 유효 (0)
        return 1 if end_date < today else 0
        
    except:
        # 변환 실패하면 무기한으로 처리
        return 0

# 적용
ads_list['expire'] = ads_list['ads_edate'].apply(check_expire)
ads_list.head()

In [ ]:
# ads_seg 테이블 만들기

click['click_date'] = pd.to_datetime(click['click_date'])

ads_seg = click.groupby('ads_idx').agg({
    'mda_idx': 'nunique',        # 매체사 수
    'dvc_idx': 'nunique',        # 유니크 사용자 수 (있다면)
    'click_key': 'count',
    'conversion': 'sum',
    'ads_category': 'first',
    'domain': 'first',
    'ads_os_type': 'first',
    'ads_order': 'first',
    'ctit': ['mean', 'median'],
    'ads_rejoin_type': 'first',
    'contract_price': 'first',
    'media_price': 'first',
    'click_date': ['min', 'max']
}).reset_index()

# 컬럼명 변경
ads_seg.columns = ['ads_idx', 'media_count', 'user_count', 'total_clicks', 'total_conversions', 'ads_category', 'domain', 'ads_os_type', 
                   'ads_order', 'ctit_mean', 'ctit_median', 'ads_rejoin_type', 'contract_price', 'media_price', 'first_click', 'last_click']

# (ads_name) 컬럼 붙이기
ads_seg = ads_seg.merge(ads_list[['ads_idx', 'ads_name', 'ads_sdate', 'expire']], on='ads_idx', how='left')

# 광고 기간 및 일평균 전환수 계산
ads_seg['days_active'] = (ads_seg['last_click'] - ads_seg['first_click']).dt.days + 1
ads_seg['daily_avg_conversions'] = ads_seg['total_conversions'] / ads_seg['days_active']


# 추후에 시간 datetime으로 바꿔서 active 여부랑 기간 넣기.

In [ ]:
# cvr 구하기
# 마진 구하기

ads_seg['cvr'] = (ads_seg['total_conversions'] / ads_seg['total_clicks']).round(1)
ads_seg['margin'] = (ads_seg['contract_price'] - ads_seg['media_price'])
ads_seg['roi'] = (ads_seg['margin'] / ads_seg['media_price']).round(1)
ads_seg['total_net_return'] = ads_seg['margin'] * ads_seg['total_conversions']


In [ ]:
# 광고 사이즈 구분

def standard_tier_classification(df):
    # daily_clicks 계산
    df['daily_clicks'] = df['total_clicks'] / df['days_active']
    
    # 점수 계산 (동일)
    def get_percentile_score(value, series):
        if value >= series.quantile(0.99): return 4
        elif value >= series.quantile(0.90): return 3  
        elif value >= series.quantile(0.70): return 2
        elif value >= series.quantile(0.40): return 1
        else: return 0
    
    df['media_score'] = df['media_count'].apply(lambda x: get_percentile_score(x, df['media_count']))
    df['conv_score'] = df['daily_avg_conversions'].apply(lambda x: get_percentile_score(x, df['daily_avg_conversions']))
    df['clicks_score'] = df['daily_clicks'].apply(lambda x: get_percentile_score(x, df['daily_clicks']))
    df['stability_score'] = df['days_active'].apply(lambda x: get_percentile_score(x, df['days_active']))
    df['cvr_score'] = df['cvr'].apply(lambda x: get_percentile_score(x, df['cvr']))
    
    df['total_score'] = (df['media_score'] + df['conv_score'] + df['clicks_score'] + 
                        df['stability_score'] + df['cvr_score'])
    
    # 표준 분류 (피라미드 형태)
    def get_standard_tier(score, score_series):
        if score >= score_series.quantile(0.995): return 'MEGA'    # 상위 5%
        elif score >= score_series.quantile(0.80): return 'LARGE'  # 상위 20%
        elif score >= score_series.quantile(0.30): return 'MEDIUM' # 상위 70% (가장 많음)
        else: return 'SMALL'                                      # 하위 30%
    
    df['ads_size'] = df['total_score'].apply(lambda x: get_standard_tier(x, df['total_score']))
    
    return df


ads_seg = standard_tier_classification(ads_seg)

print("백분위 기준 분류:")
print(ads_seg['ads_size'].value_counts())
print(f"\n분류 비율:")
print((ads_seg['ads_size'].value_counts() / len(ads_seg) * 100).round(1))
print(ads_seg.head())

### cv_goal

In [ ]:
# M, A 컬럼 붙여 넣기

# 먼저 M, A 컬럼 생성
cv_goal['M'] = (cv_goal['sch_type'] == 'M').astype(int)
cv_goal['A'] = (cv_goal['sch_type'] == 'A').astype(int)

# ads_idx 기준으로 그룹화하여 집계
cv_goal_grouped = cv_goal.groupby('ads_idx').agg({
    'mda_idx_arr': 'first',  # 첫 번째 값 사용 (M과 A가 다를 수 있으니 확인 필요)
    'sch_sdatetime': 'first',  # 시작일은 첫 번째 값
    'sch_edatetime': 'first',  # 종료일은 첫 번째 값
    'sch_adv_pay': 'first',
    'sch_ads_pay': 'first',
    'sch_mda_pay': 'max',
    'sch_clk_num': 'first',
    'sch_turn_num': 'sum',  # turn_num은 합계가 적절할 수 있음
    'M': 'max',  # M이 하나라도 있으면 1
    'A': 'max'   # A가 하나라도 있으면 1
}).reset_index()

print(f"Original rows: {len(cv_goal)}")
print(f"After grouping: {len(cv_goal_grouped)}")
print("\nM, A 조합 분포:")
print(cv_goal_grouped[['M', 'A']].value_counts())


In [ ]:
# ads_seg 랑 조인

# ads_idx를 기준으로 조인
merged_df = ads_seg.merge(
    cv_goal_grouped[['ads_idx', 'mda_idx_arr', 'M', 'A']], 
    on='ads_idx', 
    how='left'  # 또는 'inner', 'outer' 등 필요에 따라
)

# 여러 컬럼을 한 번에 처리
merged_df = merged_df.fillna({
    'mda_idx_arr': 'None',
    'M': 0,
    'A': 0
})

In [ ]:
# M==1 인 것들 중에서  광고가 우선 배정되는 m__arr 와 실제 들어가는 광고 똑같은 것 알아보기 (jaccard 사용)

m_ads = merged_df[merged_df['M']==1]

# 1) time_df에서 ads_idx별 mda_idx 리스트 만들기
time_mda = (
    time_df
      .dropna(subset=['ads_idx', 'mda_idx'])
      .groupby('ads_idx')['mda_idx']
      .unique()            # np.ndarray
      .map(list)           # list로 변환
      .rename('mda_idx_from_time')
      .reset_index()
)

# 2) m_ads와 조인
out = m_ads.merge(time_mda, on='ads_idx', how='left')

# ------------------ 여기부터 교체/추가 ------------------

def normalize_mda_list(x):
    """여러 형태(x)가 와도 [아이디, ...] 형태의 평평한 리스트로 표준화"""
    # 결측/None 처리
    if x is None or (isinstance(x, float) and pd.isna(x)) or (isinstance(x, str) and x.strip().lower() in {"none", "nan", ""}):
        return []
    # 문자열이면 안전 파싱 시도
    if isinstance(x, str):
        try:
            x = ast.literal_eval(x)
        except Exception:
            # '1,2,3' 같은 단순 콤마 문자열 대응
            return [i.strip() for i in x.split(',') if i.strip() != '']

    # 튜플은 리스트로
    if isinstance(x, tuple):
        x = list(x)

    # 리스트 계열은 내부 원소 평탄화
    if isinstance(x, list):
        flat = []
        for e in x:
            if e is None or (isinstance(e, float) and pd.isna(e)):
                continue
            if isinstance(e, tuple) or isinstance(e, list):
                flat.extend(list(e))       # [(562,563)] → [562,563], [[562,563]] → [562,563]
            else:
                flat.append(e)
        # 중복 제거(순서 보존)
        seen = set()
        dedup = []
        for v in flat:
            key = str(v)                   # 타입 혼합 대비
            if key not in seen:
                seen.add(key)
                dedup.append(v)
        return dedup

    # 그 외 스칼라
    return [x]

# 두 컬럼 모두 동일 규칙으로 표준화
out['mda_idx_arr_list']  = out['mda_idx_arr'].apply(normalize_mda_list)
out['mda_idx_from_time'] = out['mda_idx_from_time'].apply(normalize_mda_list)

# 4) 비교용: 집합(순서 무시)
to_set = lambda v: set(map(str, v))   # 타입 혼합 대비 문자열화
out['arr_set']  = out['mda_idx_arr_list'].apply(to_set)
out['time_set'] = out['mda_idx_from_time'].apply(to_set)

# 5) 비교 결과
out['is_equal_unordered'] = (out['arr_set'] == out['time_set'])
out['missing_in_arr'] = (out['time_set'] - out['arr_set']).apply(lambda s: sorted(s))
out['extra_in_arr']   = (out['arr_set'] - out['time_set']).apply(lambda s: sorted(s))

def jaccard(a, b):
    if not a and not b:
        return 1.0
    return len(a & b) / len(a | b) if (a | b) else 0.0

out['jaccard'] = [jaccard(a, b) for a, b in zip(out['arr_set'], out['time_set'])]

# 우선순위 1순위가 실제 등장하는지
out['top_priority_in_time'] = [
    (str(lst[0]) in tset) if lst else False
    for lst, tset in zip(out['mda_idx_arr_list'], out['time_set'])
]

# 6) 보기용 요약
compare_cols = [
    'ads_idx', 'mda_idx_arr_list', 'mda_idx_from_time',
    'is_equal_unordered', 'missing_in_arr', 'extra_in_arr',
    'jaccard', 'top_priority_in_time'
]
result_compare = out[compare_cols]



In [ ]:
# 지정매체광고
# mda_idx_from_time이 mda_idx_arr_list에 포함되거나 같은것만 뽑는거

mask = [
    all(m in row['mda_idx_arr_list'] for m in row['mda_idx_from_time'])
    for _, row in result_compare.iterrows()
]
restricted_mda = result_compare[mask]

restricted_mda

### ads_pool

In [ ]:
# 지정매체광고x
# 이거 반지정 아예 아닌거 나눠야하나?

# 제외할 광고 아이디 (restricted_mda에서)
exclude_ids = set(restricted_mda['ads_idx'].unique())

# ads_seg에서 이 아이디들 아닌 행만 보기
ads_seg_filter = merged_df[ ~merged_df['ads_idx'].isin(exclude_ids) ].copy()

# (선택) 인덱스 리셋
ads_seg_filter = ads_seg_filter.reset_index(drop=True)

In [ ]:
# expire 안된 애들 골라내기

ads_pool= ads_seg_filter[ads_seg_filter['expire'] == 0]

## media_portfolio

In [ ]:
# 1단계: 매체사-광고 관계 추출
media_ads_mapping = click[['mda_idx', 'ads_idx']].drop_duplicates()

# 2단계: 광고 분류 정보 조인
media_ads_mapping = media_ads_mapping.merge(
    ads_seg[['ads_idx', 'ads_size']], 
    on='ads_idx', 
    how='left'
)

# 3단계: 매체사별 광고 레벨 집계
media_portfolio = media_ads_mapping.groupby(['mda_idx', 'ads_size']).size().unstack(fill_value=0)

# 4단계: 총 광고 수 및 비율 계산
media_portfolio['total_ads'] = media_portfolio.sum(axis=1)

# 각 레벨별 비율 계산
for level in ['MEGA', 'LARGE', 'MEDIUM', 'SMALL']:
    if level in media_portfolio.columns:
        media_portfolio[f'{level}_ratio'] = (
            media_portfolio[level] / media_portfolio['total_ads'] * 100
        ).round(1)

# 5단계: 결과 확인
print("매체사별 포트폴리오 (상위 10개):")
print(media_portfolio.sort_values('total_ads', ascending=False).head(10))

# 6단계: 매체사 특성 분석
print("\n매체사 포트폴리오 요약:")
print(f"총 매체사 수: {len(media_portfolio)}")
print(f"평균 광고 보유 수: {media_portfolio['total_ads'].mean():.1f}")
print(f"광고 10개 이상 보유 매체사: {(media_portfolio['total_ads'] >= 10).sum()}개")

In [ ]:
# 1. 매체사 기본 정보 
media_basic = click.groupby('mda_idx').agg({
    'dvc_idx': 'nunique',        
    'click_key': 'count',
    'conversion': 'sum',
    'click_date': ['min', 'max']
}).reset_index()

media_basic.columns = ['mda_idx', 'user_count', 'total_clicks', 'total_conversions', 'first_click', 'last_click']

# 기간 및 일평균 계산
media_basic['days_active'] = (media_basic['last_click'] - media_basic['first_click']).dt.days + 1
media_basic['daily_avg_conversions'] = media_basic['total_conversions'] / media_basic['days_active']

# 2. 매체사-광고 관계
media_ads_mapping = click[['mda_idx', 'ads_idx']].drop_duplicates()
media_ads_mapping = media_ads_mapping.merge(
    ads_seg[['ads_idx', 'ads_size']], 
    on='ads_idx', 
    how='left'
)

# 3. 포트폴리오 구성
media_portfolio_composition = media_ads_mapping.groupby(['mda_idx', 'ads_size']).size().unstack(fill_value=0)
media_portfolio_composition['total_ads'] = media_portfolio_composition.sum(axis=1)

# 비율 계산
for level in ['MEGA', 'LARGE', 'MEDIUM', 'SMALL']:
    if level in media_portfolio_composition.columns:
        media_portfolio_composition[f'{level}_ratio'] = (
            media_portfolio_composition[level] / media_portfolio_composition['total_ads'] * 100
        ).round(1)

# 4. 최종 합치기
media_portfolio = media_basic.merge(
    media_portfolio_composition, 
    on='mda_idx', 
    how='left'
)

media_portfolio


In [ ]:
# 1. 매체사-광고 관계에 광고 속성 정보 추가
media_ads_with_attributes = media_ads_mapping.merge(
    ads_list[['ads_idx', 'ads_category', 'ads_os_type', 'domain']], 
    on='ads_idx', 
    how='left'
)

# 2. 각 속성별로 매체사별 분포 계산
def calculate_attribute_distribution(df, attribute_col):
    # 매체사별 해당 속성 분포
    attr_dist = df.groupby(['mda_idx', attribute_col]).size().unstack(fill_value=0)
    
    # 총 광고 수로 나누어 비율 계산
    attr_dist_pct = attr_dist.div(attr_dist.sum(axis=1), axis=0) * 100
    
    # 컬럼명에 prefix 추가
    attr_dist_pct.columns = [f'{attribute_col}_{col}_pct' for col in attr_dist_pct.columns]
    
    return attr_dist_pct

# 3. 각 속성별 비율 계산
category_dist = calculate_attribute_distribution(media_ads_with_attributes, 'ads_category')
os_dist = calculate_attribute_distribution(media_ads_with_attributes, 'ads_os_type')  
domain_dist = calculate_attribute_distribution(media_ads_with_attributes, 'domain')

# 4. 모든 분포 정보를 기존 테이블에 조인
media_portfolio = media_portfolio.merge(
    category_dist, on='mda_idx', how='left'
).merge(
    os_dist, on='mda_idx', how='left'
).merge(
    domain_dist, on='mda_idx', how='left'
)

# 5. 결과 확인
print(f"최종 테이블 크기: {media_portfolio.shape}")
print("추가된 컬럼들:")
new_cols = [col for col in media_portfolio.columns if '_pct' in col]
print(new_cols[:10])  # 처음 10개만 출력

# 샘플 확인
print("\n매체사별 속성 분포 (상위 5개 매체사):")
sample_cols = ['mda_idx', 'total_ads'] + new_cols[:6]  # 샘플로 몇 개만
print(media_portfolio[sample_cols].head())

In [ ]:
# 1. click 데이터에서 순수익 계산
click['profit_per_conversion'] = click['contract_price'] - click['media_price']

# 2. 각 행별 실제 수익 계산 (전환이 발생한 경우만)
click['actual_profit'] = click['profit_per_conversion'] * click['conversion']

# 3. 매체사별 총 예상 수익 집계
media_expected_profit = click.groupby('mda_idx')['actual_profit'].sum().reset_index()
media_expected_profit.columns = ['mda_idx', 'expected_total_profit']

# 4. media_portfolio에 조인
media_portfolio = media_portfolio.merge(
    media_expected_profit, 
    on='mda_idx', 
    how='left'
)

# NaN 처리 (수익이 없는 매체사는 0)
media_portfolio['expected_total_profit'] = (
    media_portfolio['expected_total_profit'].fillna(0)
)

# 결과 확인
print("매체사별 예상 수익 상위 10개:")
top_profit = media_portfolio.sort_values('expected_total_profit', ascending=False)
print(top_profit[['mda_idx', 'total_ads', 'total_conversions', 'expected_total_profit']].head(10))

print(f"\n전체 매체사 예상 수익 합계: {media_portfolio['expected_total_profit'].sum():,.0f}")

### media_group

In [ ]:

def classify_media_companies(df, auto_thresholds=False, reduce_misc=True):
    """
    매체사를 6가지 유형으로 분류 (기타 최소화 옵션 포함)
    """
    THRESHOLDS = {
        'total_ads_large': 115,        # 상위 10% (대량처리)
        'total_ads_medium': 28,        # 상위 25% (안정공급)
        'daily_conv_high': 500,        # 상위 10% (고성능)
        'daily_conv_medium': 164,      # 상위 25% (중성능)
        'days_active_stable': 25,      # 안정적 활동
        'days_active_new': 15,         # 신규 기준
        'conversion_rate_low': 0.001,  # 어뷰징 의심(0.1%)
        'specialization_threshold': 50, # 도메인 특화 (%)
        'mega_specialization': 70,     # MEGA 특화
        'large_specialization': 80,    # LARGE 특화
        'min_performance': 10,         # 최소 성과
        'inactive_cutoff': '2025-08-23' # 비활성 기준일(8/23 포함 이전)
    }
    ABUSE_MIN_CLICKS = 1000

    media_df = df.copy()

    # 파생 지표
    media_df['conversion_rate'] = media_df['total_conversions'] / media_df['total_clicks'].replace(0, np.nan)
    media_df['conversion_rate'] = media_df['conversion_rate'].fillna(0)
    media_df['last_click_dt'] = pd.to_datetime(media_df['last_click'], errors='coerce')
    cutoff_date = pd.to_datetime(THRESHOLDS['inactive_cutoff'])

    # (선택) 분위수 기반 자동 튜닝
    def _maybe_autotune_thresholds(df, TH, enabled):
        if not enabled: return TH
        q = df.quantile
        tuned = TH.copy()
        tuned.update({
            'total_ads_large': int(q(0.90)['total_ads']),
            'total_ads_medium': int(q(0.75)['total_ads']),
            'daily_conv_high': float(q(0.90)['daily_avg_conversions']),
            'daily_conv_medium': float(q(0.75)['daily_avg_conversions']),
            'days_active_stable': max(14, int(q(0.75)['days_active'])),
            'days_active_new': int(q(0.25)['days_active']),
            'conversion_rate_low': max(0.001, float(df['conversion_rate'].quantile(0.05))),
        })
        return tuned

    THRESHOLDS = _maybe_autotune_thresholds(media_df, THRESHOLDS, auto_thresholds)

    def check_domain_specialization(row):
        cols = [c for c in media_df.columns if c.startswith('domain_') and c.endswith('_pct')]
        for col in cols:
            v = row.get(col, np.nan)
            if pd.notna(v) and v >= THRESHOLDS['specialization_threshold']:
                name = col.replace('domain_', '').replace('_pct', '')
                return f"{name}특화"
        return None

    def check_size_specialization(row):
        if pd.notna(row.get('MEGA_ratio')) and row['MEGA_ratio'] >= THRESHOLDS['mega_specialization']:
            return "MEGA특화"
        if pd.notna(row.get('LARGE_ratio')) and row['LARGE_ratio'] >= THRESHOLDS['large_specialization']:
            return "LARGE특화"
        return None

    def classify_single_media(row):
        # 1) 계약종료형 (최우선)
        if (row['days_active'] <= 7) or (pd.notna(row['last_click_dt']) and row['last_click_dt'] < cutoff_date):
            return "계약종료형"

        # 2) 품질관리형 (어뷰징 의심)
        if (row['total_clicks'] >= ABUSE_MIN_CLICKS) and (row['conversion_rate'] < THRESHOLDS['conversion_rate_low']):
            return "품질관리형"

        # 3) 대량처리형
        if (row['total_ads'] >= THRESHOLDS['total_ads_large'] and
            row['daily_avg_conversions'] >= THRESHOLDS['daily_conv_high'] and
            row['days_active'] >= THRESHOLDS['days_active_stable']):
            return "대량처리형"

        # 4) 특화전문형
        domain_spec = check_domain_specialization(row)
        size_spec = check_size_specialization(row)
        if (domain_spec or size_spec) and (row['days_active'] >= THRESHOLDS['days_active_new']):
            return f"특화전문형_{domain_spec or size_spec}"

        # 5) 신규개발형
        if (row['days_active'] <= THRESHOLDS['days_active_new'] and
            row['daily_avg_conversions'] >= THRESHOLDS['min_performance']):
            return "신규개발형"

        # 6) 안정공급형
        if (row['total_ads'] >= THRESHOLDS['total_ads_medium'] and
            row['daily_avg_conversions'] >= THRESHOLDS['daily_conv_medium'] and
            row['days_active'] >= THRESHOLDS['days_active_stable']):
            return "안정공급형"

        # ---- Fallback: 기타 줄이기 ----
        if reduce_misc:
            # (a) 특화 신호만 있는 경우 → 특화전문형(후보)
            if (domain_spec or size_spec):
                return f"특화전문형_{domain_spec or size_spec}(후보)"
            # (b) 거의 안정 조건에 근접 → 안정공급형(후보)
            if (row['days_active'] >= max(THRESHOLDS['days_active_new'], THRESHOLDS['days_active_stable'] - 5)) and \
               ((row['daily_avg_conversions'] >= THRESHOLDS['min_performance']) or
                (row['total_ads'] >= int(THRESHOLDS['total_ads_medium'] * 0.6))):
                return "안정공급형(후보)"
            # (c) 활동일이 짧으면 → 신규개발형
            if row['days_active'] <= THRESHOLDS['days_active_new']:
                return "신규개발형"
            # (d) 남은 케이스는 관리 필요로 흡수
            return "관리 필요"

        return "기타"

    media_df['classification'] = media_df.apply(classify_single_media, axis=1)
    media_df['basic_classification'] = media_df['classification'].apply(lambda x: x.split('_')[0] if '_' in x else x)

    return media_df


In [ ]:
mda_pf_class = classify_media_companies(media_portfolio)

In [ ]:
# mda_pf_class에서 매체사 980번 빼기
# 너무 수익 이런거 혼자 극단치 

mda_pf_class = mda_pf_class[mda_pf_class['mda_idx'] != 980].reset_index(drop=True)

In [ ]:
# 한글, 오류 해결

def setup_korean_font(preferred=None, verbose=True):
    # 우선순위 높은 후보들(설치된 것 중 첫 번째를 씁니다)
    candidates = (preferred or []) + [
        'AppleGothic',            # macOS
        'Malgun Gothic',          # Windows
        'NanumGothic', 'NanumBarunGothic', 'NanumGothicCoding',
        'Noto Sans CJK KR', 'Noto Sans KR',
        'Arial Unicode MS'
    ]
    available = {f.name for f in font_manager.fontManager.ttflist}
    for name in candidates:
        if name in available:
            rcParams['font.family'] = name
            rcParams['axes.unicode_minus'] = False  # 마이너스 기호 깨짐 방지
            if verbose:
                print(f'✅ Using font: {name}')
            break
    else:
        print('⚠️ 한글 폰트가 발견되지 않았습니다. NanumGothic 또는 Noto Sans KR 설치 후 다시 실행하세요.')

def silence_matplotlib_warnings():
    # 글리프 경고/기타 matplotlib 경고 숨기기
    warnings.filterwarnings("ignore", message=r"Glyph .* missing from font", category=UserWarning)
    warnings.filterwarnings("ignore", category=UserWarning, module="matplotlib")
    logging.getLogger('matplotlib').setLevel(logging.ERROR)

setup_korean_font()
silence_matplotlib_warnings()

In [ ]:

DF = mda_pf_class.copy()  # classify_media_companies(mda_pf) 결과

# 분류 라벨 정렬(보고서용 가독성)
preferred_order = [
    '대량처리형', '안정공급형', '안정공급형(후보)',
    '특화전문형', '신규개발형',
    '관리 필요', '품질관리형', '계약종료형'
]
classes_in_df = [c for c in preferred_order if c in DF['basic_classification'].unique()]
DF['basic_classification'] = pd.Categorical(DF['basic_classification'], classes_in_df, ordered=True)

# 사용할 지표 목록
metrics = {
    'expected_total_profit': '기대 수익',
    'user_count': '유저 수',
    'total_ads': '광고 수',
    'daily_avg_conversions': '일평균 전환',
    'conversion_rate': '전환율',
    'MEGA_ratio': 'MEGA 비중(%)',
    'LARGE_ratio': 'LARGE 비중(%)',
    'days_active': '활동 일수'
}

# 요약 테이블(개수/평균/중앙값/합계)
summary = DF.groupby('basic_classification').agg(
    count=('mda_idx','count'),
    expected_total_profit_sum=('expected_total_profit','sum'),
    expected_total_profit_mean=('expected_total_profit','mean'),
    expected_total_profit_median=('expected_total_profit','median'),
    user_count_mean=('user_count','mean'),
    user_count_median=('user_count','median'),
    total_ads_mean=('total_ads','mean'),
    daily_avg_conversions_mean=('daily_avg_conversions','mean'),
    conversion_rate_mean=('conversion_rate','mean'),
    MEGA_ratio_mean=('MEGA_ratio','mean'),
    LARGE_ratio_mean=('LARGE_ratio','mean'),
    days_active_mean=('days_active','mean')
).round(2)
display(summary)  # 노트북 환경이면 표로 보임

# ---------- 헬퍼 함수 ----------
def _bar(series, title, ylabel, rotate=0, fmt_pct=False):
    plt.figure()
    ax = series.plot(kind='bar')
    ax.set_title(title)
    ax.set_ylabel(ylabel)
    ax.set_xlabel('분류')
    if rotate:
        plt.xticks(rotation=rotate)
    if fmt_pct:
        # y축 퍼센트 표시
        vals = ax.get_yticks()
        ax.set_yticklabels([f"{v:.0f}%" for v in vals])
    plt.tight_layout()
    plt.show()

def _box_by_class(df, col, title, ylabel, clip_q=0.99, log=False):
    dat = df[['basic_classification', col]].dropna().copy()
    # 극단값 클리핑(박스플롯 스케일 안정화)
    upper = dat[col].quantile(clip_q)
    dat[col] = np.clip(dat[col], dat[col].min(), upper)
    plt.figure()
    # 하나의 축 안에서 카테고리별 박스플롯
    groups = [g[col].values for _, g in dat.groupby('basic_classification')]
    labels = [str(c) for c in dat['basic_classification'].cat.categories]
    plt.boxplot(groups, labels=labels, showfliers=False)
    plt.title(title)
    plt.ylabel(ylabel)
    if log:
        plt.yscale('log')
    plt.xticks(rotation=20)
    plt.tight_layout()
    plt.show()

def _scatter(df, x, y, title, xlabel, ylabel, size_col=None, alpha=0.5, logx=False, logy=False):
    dat = df[[x, y, 'basic_classification'] + ([size_col] if size_col else [])].dropna().copy()
    plt.figure()
    if size_col:
        # 마커 크기 표준화
        s = dat[size_col].replace(0, np.nan)
        s = 50 * (s / s.quantile(0.9)).clip(0.2, 3)  # 적당한 범위로 스케일
        plt.scatter(dat[x], dat[y], s=s, alpha=alpha)
    else:
        plt.scatter(dat[x], dat[y], alpha=alpha)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    if logx: plt.xscale('log')
    if logy: plt.yscale('log')
    plt.tight_layout()
    plt.show()

# ---------- 1) 분류 분포(개수/비율) ----------
counts = DF['basic_classification'].value_counts().reindex(classes_in_df).fillna(0)
_bar(counts, '분류별 매체사 개수', '개수', rotate=20)

pct = (counts / counts.sum() * 100).round(1)
_bar(pct, '분류별 매체사 비율', '비율(%)', rotate=20, fmt_pct=True)

# ---------- 2) 분류별 기대수익 ----------
# 합계(시장 기여도)
profit_sum = DF.groupby('basic_classification')['expected_total_profit'].sum().reindex(classes_in_df)
_bar(profit_sum, '분류별 기대수익 합계', '기대수익 합계', rotate=20)

# 평균/중앙값(단위 매체 기대수익)
profit_mean = DF.groupby('basic_classification')['expected_total_profit'].mean().reindex(classes_in_df)
_bar(profit_mean, '분류별 기대수익 평균', '평균 기대수익', rotate=20)

profit_median = DF.groupby('basic_classification')['expected_total_profit'].median().reindex(classes_in_df)
_bar(profit_median, '분류별 기대수익 중앙값', '중앙 기대수익', rotate=20)

# 분포(대수 스케일 권장)
_box_by_class(DF, 'expected_total_profit', '분류별 기대수익 분포(박스플롯)', '기대수익', clip_q=0.99, log=True)

# ---------- 3) 유저 수 ----------
user_mean = DF.groupby('basic_classification')['user_count'].mean().reindex(classes_in_df)
_bar(user_mean, '분류별 유저 수 평균', '평균 유저 수', rotate=20)

_box_by_class(DF, 'user_count', '분류별 유저 수 분포(박스플롯)', '유저 수', clip_q=0.99, log=True)

# ---------- 4) 운영 볼륨(광고 수, 일평균 전환) ----------
ads_mean = DF.groupby('basic_classification')['total_ads'].mean().reindex(classes_in_df)
_bar(ads_mean, '분류별 광고 수 평균', '평균 광고 수', rotate=20)

_box_by_class(DF, 'daily_avg_conversions', '분류별 일평균 전환 분포(박스플롯)', '일평균 전환', clip_q=0.99, log=True)

# ---------- 5) 전환율 ----------
cr_mean = DF.groupby('basic_classification')['conversion_rate'].mean().reindex(classes_in_df)
_bar(cr_mean, '분류별 전환율 평균', '평균 전환율', rotate=20)

_box_by_class(DF, 'conversion_rate', '분류별 전환율 분포(박스플롯)', '전환율', clip_q=0.99, log=False)

# ---------- 6) 포맷 특성(MEGA/LARGE 비중) ----------
mega_mean = DF.groupby('basic_classification')['MEGA_ratio'].mean().reindex(classes_in_df)
_bar(mega_mean, '분류별 MEGA 비중 평균(%)', 'MEGA 비중(%)', rotate=20)

large_mean = DF.groupby('basic_classification')['LARGE_ratio'].mean().reindex(classes_in_df)
_bar(large_mean, '분류별 LARGE 비중 평균(%)', 'LARGE 비중(%)', rotate=20)

# ---------- 7) 활동성 ----------
days_mean = DF.groupby('basic_classification')['days_active'].mean().reindex(classes_in_df)
_bar(days_mean, '분류별 활동 일수 평균', '평균 활동 일수', rotate=20)

_box_by_class(DF, 'days_active', '분류별 활동 일수 분포(박스플롯)', '활동 일수', clip_q=0.99, log=False)

# ---------- 8) 관계형 시각화(산점도) ----------
# 기대수익 vs 유저 수 (마커 크기: 광고 수)
_scatter(DF, 'user_count', 'expected_total_profit',
         '기대수익 vs 유저 수 (마커 크기=광고 수)',
         '유저 수', '기대수익', size_col='total_ads', logx=True, logy=True)

# 기대수익 vs 일평균 전환
_scatter(DF, 'daily_avg_conversions', 'expected_total_profit',
         '기대수익 vs 일평균 전환', '일평균 전환', '기대수익', logx=True, logy=True)

# 전환율 vs 기대수익
_scatter(DF, 'conversion_rate', 'expected_total_profit',
         '기대수익 vs 전환율', '전환율', '기대수익', logx=False, logy=True)


# 매체사별 광고 성과

In [ ]:
def analyze_ads_performance(ads_idx, click_data, media_portfolio=None):
    """
    특정 광고의 매체별 성과를 분석하는 함수
    """
    
    # 1. 해당 광고의 데이터가 있는지 확인
    ads_data = click_data[click_data['ads_idx'] == ads_idx]
    if len(ads_data) == 0:
        print(f"광고 {ads_idx}에 대한 데이터가 없습니다.")
        return pd.DataFrame()
    
    # 2. 기본 성과 데이터 추출
    ads_performance = ads_data.groupby(['ads_idx', 'mda_idx']).agg({
        'click_key': 'count',
        'conversion': 'sum',
        'contract_price': 'first',
        'media_price': 'first',
        'domain': 'first',
        'ads_category': 'first'
    }).reset_index()
    
    # 컬럼명 변경
    ads_performance.columns = ['ads_idx', 'mda_idx', 'total_clicks', 'total_conversions', 
                              'contract_price', 'media_price', 'domain', 'ads_category']
    
    # 전환율 및 수익 계산
    ads_performance['cvr'] = (
        ads_performance['total_conversions'] / ads_performance['total_clicks']
    ).round(4)
    
    ads_performance['profit_per_conversion'] = (
        ads_performance['contract_price'] - ads_performance['media_price']
    )
    ads_performance['total_profit'] = (
        ads_performance['total_conversions'] * ads_performance['profit_per_conversion']
    )
    
    # 3. 날짜 범위 및 활동일 계산
    click_data_copy = click_data.copy()
    if not pd.api.types.is_datetime64_any_dtype(click_data_copy['click_date']):
        click_data_copy['click_date'] = pd.to_datetime(click_data_copy['click_date'])
    
    ads_activity = (
        click_data_copy.loc[click_data_copy['ads_idx'] == ads_idx]
                      .groupby('mda_idx')['click_date']
                      .agg(first_click='min', last_click='max')
                      .reset_index()
    )
    
    ads_activity['days_active_calc'] = (
        (ads_activity['last_click'] - ads_activity['first_click']).dt.days + 1
    )
    
    # 4. 데이터 병합
    merged = ads_performance.merge(
        ads_activity[['mda_idx', 'first_click', 'last_click', 'days_active_calc']],
        on='mda_idx', how='left'
    )
    
    # 5. 일평균 지표 계산
    merged['daily_clicks'] = merged['total_clicks'] / merged['days_active_calc']
    merged['daily_conversions'] = merged['total_conversions'] / merged['days_active_calc']
    merged['daily_profit'] = merged['total_profit'] / merged['days_active_calc']
    
    # 6. 배분 그룹 분류 (데이터가 충분한 경우에만)
    if len(merged) > 1:  # 최소 2개 이상의 매체가 있어야 중앙값 계산이 의미있음
        profit_median = merged['daily_profit'].median()
        conv_median = merged['daily_conversions'].median()
        
        merged['배분그룹'] = np.where(
            (merged['daily_profit'] >= profit_median) & (merged['daily_conversions'] >= conv_median),
            '잘 배분',
            '잘못 배분'
        )
        # 결과 정렬
        result = merged.sort_values(['배분그룹', 'daily_profit'], ascending=[True, False]).reset_index(drop=True)
    else:
        merged['배분그룹'] = '분류불가'
        result = merged.reset_index(drop=True)
    
    return result

# 매체사 유사도

In [ ]:
# 매체사 포트폴리오 각 매체사별 카테고리나 도메인 전환 비중 


# 컬럼 이름에서 공백/특수문자 → '_' 로 바꾸는 헬퍼
def _slug(s):
    return re.sub(r'[^0-9A-Za-z가-힣]+', '_', str(s)).strip('_')

def add_cat_domain_to_mda_pf(
    mda_pf: pd.DataFrame,
    clicks_df: pd.DataFrame,
    conv_col: str = "conversion",
    cat_col: str = "ads_category",
    dom_col: str = "domain",
    add_within_cat: bool = False,      # mda×카테고리 내부 도메인 구성비 추가 여부
    add_within_dom: bool = False       # mda×도메인 내부 카테고리 구성비 추가 여부
):
    """
    반환: (enriched_mda_pf, new_columns)
    - conv_cat{카테고리}_{도메인} : 해당 mda의 (카테고리×도메인) 전환수
    - share_cat{카테고리}_{도메인}: 해당 mda 전체 전환 대비 구성비(0~1)
    - (옵션) shareWithinCat_*, shareWithinDomain_* 도 함께 추가 가능
    """
    df = clicks_df.copy()

    # 전환수 정리 (0/1이 아니면 그대로 합산, 0/1이면 1 합산)
    df[conv_col] = pd.to_numeric(df[conv_col], errors="coerce").fillna(0)
    if df[conv_col].max() <= 1:
        df["conv"] = (df[conv_col] > 0).astype(int)
    else:
        df["conv"] = df[conv_col]

    # 전환 있는 행만
    conv = df[df["conv"] > 0].copy()
    if conv.empty:
        enriched = mda_pf.copy()
        return enriched, []

    # mda × category × domain 전환수 집계
    g = (conv.groupby(["mda_idx", cat_col, dom_col], as_index=False)["conv"]
              .sum())

    # mda 전체 전환 합 → mda 대비 구성비
    total_mda = (g.groupby("mda_idx", as_index=False)["conv"]
                   .sum()
                   .rename(columns={"conv":"total_mda"}))
    g = g.merge(total_mda, on="mda_idx", how="left")
    g["share_mda"] = g["conv"] / g["total_mda"].replace(0, np.nan)
    g["share_mda"] = g["share_mda"].fillna(0.0)

    # ---- 피벗: 전환수 / mda-구성비
    piv_cnt = (g.pivot(index="mda_idx",
                       columns=[cat_col, dom_col],
                       values="conv")
                 .fillna(0))
    piv_shr = (g.pivot(index="mda_idx",
                       columns=[cat_col, dom_col],
                       values="share_mda")
                 .fillna(0.0))

    # 컬럼 평탄화
    piv_cnt.columns = [f"conv_cat{c}_{_slug(d)}" for c, d in piv_cnt.columns]
    piv_shr.columns = [f"share_cat{c}_{_slug(d)}" for c, d in piv_shr.columns]

    out = (mda_pf.merge(piv_cnt, on="mda_idx", how="left")
                 .merge(piv_shr, on="mda_idx", how="left"))

    new_cols = list(piv_cnt.columns) + list(piv_shr.columns)
    out[new_cols] = out[new_cols].fillna(0)

    # ---- (옵션) mda×카테고리 내부 도메인 구성비
    if add_within_cat:
        tot_cat = (g.groupby(["mda_idx", cat_col], as_index=False)["conv"]
                     .sum()
                     .rename(columns={"conv":"_tot_cat"}))
        g2 = g.merge(tot_cat, on=["mda_idx", cat_col], how="left")
        g2["share_within_cat"] = g2["conv"] / g2["_tot_cat"].replace(0, np.nan)
        piv_wc = (g2.pivot(index="mda_idx",
                           columns=[cat_col, dom_col],
                           values="share_within_cat")
                    .fillna(0.0))
        piv_wc.columns = [f"shareWithinCat_cat{c}_{_slug(d)}" for c, d in piv_wc.columns]
        out = out.merge(piv_wc, on="mda_idx", how="left")
        out[piv_wc.columns] = out[piv_wc.columns].fillna(0.0)
        new_cols += list(piv_wc.columns)

    # ---- (옵션) mda×도메인 내부 카테고리 구성비
    if add_within_dom:
        tot_dom = (g.groupby(["mda_idx", dom_col], as_index=False)["conv"]
                     .sum()
                     .rename(columns={"conv":"_tot_dom"}))
        g3 = g.merge(tot_dom, on=["mda_idx", dom_col], how="left")
        g3["share_within_domain"] = g3["conv"] / g3["_tot_dom"].replace(0, np.nan)
        piv_wd = (g3.pivot(index="mda_idx",
                           columns=[cat_col, dom_col],
                           values="share_within_domain")
                    .fillna(0.0))
        piv_wd.columns = [f"shareWithinDomain_cat{c}_{_slug(d)}" for c, d in piv_wd.columns]
        out = out.merge(piv_wd, on="mda_idx", how="left")
        out[piv_wd.columns] = out[piv_wd.columns].fillna(0.0)
        new_cols += list(piv_wd.columns)

    return out, new_cols


In [ ]:
# clicks_df: 원본 클릭/전환 테이블 (mda_idx, ads_category, domain, conversion 포함)
# mda_pf: 매체 프로필 테이블 (mda_idx 기준)

mda_pf_enriched, added_cols = add_cat_domain_to_mda_pf(
    mda_pf, click,
    add_within_cat=False,     # 필요하면 True
    add_within_dom=False      # 필요하면 True
)

print(f"추가된 컬럼 수: {len(added_cols)}")
# mda_pf_enriched.head()


## 기존 광고

In [ ]:
# === 통합 셀: 유사도 추천기 (CLR + prior + power + IDF + 추가 비율 피처) ===
import numpy as np
import pandas as pd
import re

# --- 유틸 ---
def _slug(s): 
    return re.sub(r'[^0-9A-Za-z가-힣]+', '_', str(s)).strip('_')

def _cosine(a, B):
    a = a.reshape(1, -1)
    num = (B * a).sum(axis=1)
    den = (np.sqrt((B**2).sum(axis=1)) * np.sqrt((a**2).sum()))
    den = np.where(den == 0, 1e-12, den)
    return (num / den).ravel()

def cosine_vec(a, B):
    return _cosine(np.asarray(a, dtype=float), np.asarray(B, dtype=float))

# --- 광고 데이터로 (카테고리×도메인) 가중치 + prior 스무딩 ---
def make_ad_pair_weights_from_ad_df(
    ad_df, cat_col='ads_category', dom_col='domain',
    conv_col='total_conversions', power=1.0, min_frac=0.0,
    prior_mix=0.0, prior_bg=None
):
    """
    (카테×도메인) 전환 분포 -> share_cat{c}_{slug} 가중치 dict
    power<1: 퍼짐, >1: 집중 / prior_mix: 배경분포 섞기
    """
    t = ad_df.copy()
    t[conv_col] = pd.to_numeric(t[conv_col], errors='coerce').fillna(0.0)
    g = (t.groupby([cat_col, dom_col])[conv_col].sum()
           .rename('conv').reset_index())
    tot = g['conv'].sum()
    if tot <= 0:
        return {}
    g['frac'] = g['conv'] / tot
    if min_frac > 0:
        g = g[g['frac'] >= min_frac].copy()
    g['w'] = (g['frac'] ** power)
    s = g['w'].sum()
    if s > 0:
        g['w'] = g['w'] / s
    g['key'] = [f"share_cat{int(c)}_{_slug(d)}" for c,d in g[[cat_col, dom_col]].itertuples(index=False)]
    w = dict(zip(g['key'], g['w']))

    # prior 스무딩
    if prior_mix and prior_mix > 0:
        if (prior_bg is None) or (len(prior_bg) == 0):
            prior_bg = {k: 1.0/len(w) for k in w.keys()}
        keys = set(w) | set(prior_bg)
        out = {}
        for k in keys:
            pw = w.get(k, 0.0)
            q  = prior_bg.get(k, 0.0)
            out[k] = (1.0 - prior_mix) * pw + prior_mix * q
        Z = sum(out.values()) or 1.0
        w = {k: v/Z for k,v in out.items()}
    return w

# --- 구성비 CLR 변환 ---
def _clr_block(df_block, eps=1e-6):
    Z = df_block.clip(lower=eps)
    g = np.exp(np.log(Z).mean(axis=1))
    return np.log(Z.div(g, axis=0))

# --- 피처 행렬 (share + 볼륨 + 각종 비율 + CLR + 가중 + z-score) ---
def build_feature_matrix_plus(
    mda_pf,
    share_cols,                 
    volume_cols=None,           
    size_ratio_cols=None,       
    os_ratio_cols=None,         
    category_ratio_cols=None,   
    domain_ratio_cols=None,     
    use_clr=True,               
    col_weights=None,           
    zscore=True
):
    volume_cols         = list(volume_cols or [])
    size_ratio_cols     = list(size_ratio_cols or [])
    os_ratio_cols       = list(os_ratio_cols or [])
    category_ratio_cols = list(category_ratio_cols or [])
    domain_ratio_cols   = list(domain_ratio_cols or [])

    all_cols = (list(share_cols) + volume_cols + size_ratio_cols +
                os_ratio_cols + category_ratio_cols + domain_ratio_cols)

    X = mda_pf.set_index('mda_idx')[all_cols].astype(float).copy()

    # 결측
    X[volume_cols] = X[volume_cols].fillna(0.0)
    X[size_ratio_cols + os_ratio_cols + category_ratio_cols + domain_ratio_cols + share_cols] = \
        X[size_ratio_cols + os_ratio_cols + category_ratio_cols + domain_ratio_cols + share_cols].fillna(0.0)

    # 볼륨: log1p
    if volume_cols:
        X[volume_cols] = np.log1p(X[volume_cols])

    # 비율: CLR
    if use_clr:
        if size_ratio_cols:
            X[size_ratio_cols] = _clr_block(X[size_ratio_cols])
        if os_ratio_cols:
            X[os_ratio_cols] = _clr_block(X[os_ratio_cols])
        if category_ratio_cols:
            X[category_ratio_cols] = _clr_block(X[category_ratio_cols])
        if domain_ratio_cols:
            X[domain_ratio_cols] = _clr_block(X[domain_ratio_cols])

    # 열 가중
    if col_weights:
        w = pd.Series({c: col_weights.get(c, 1.0) for c in all_cols}, index=all_cols, dtype=float)
        X = X.mul(w, axis=1)

    # 표준화
    if zscore:
        X = (X - X.mean()) / (X.std() + 1e-9)

    return X, all_cols

# --- 메인 추천 ---
def recommend_with_weighted_similarity(
    ad_df,
    mda_pf,
    top_anchor_by='total_conversions',
    n_anchor=5,
    topN=20,
    weight_power=0.5,
    min_pair_frac=0.0,
    top_weight_feats=None,
    exclude_classes=('계약종료형','품질관리형'),
    min_days_active=7,
    blend_pred_table=None,
    blend_ad_id=None,
    blend_alpha=0.7,
    sort_by="final",

    # 피처 세트(있으면 자동 사용)
    volume_cols=("user_count","total_clicks","total_conversions","daily_avg_conversions","total_ads"),
    size_ratio_cols=("MEGA_ratio","LARGE_ratio","MEDIUM_ratio","SMALL_ratio"),
    os_ratio_cols=("ads_os_type_1_pct","ads_os_type_2_pct","ads_os_type_3_pct","ads_os_type_7_pct"),
    category_ratio_cols=("ads_category_0_pct","ads_category_1_pct","ads_category_2_pct","ads_category_3_pct",
                         "ads_category_4_pct","ads_category_5_pct","ads_category_6_pct","ads_category_7_pct",
                         "ads_category_8_pct","ads_category_10_pct","ads_category_11_pct","ads_category_13_pct"),
    domain_ratio_cols=("domain_게임_pct","domain_교육_pct","domain_금융_pct","domain_기타_pct","domain_미디어/컨텐츠_pct",
                       "domain_뷰티_pct","domain_비영리/공공_pct","domain_생활_pct","domain_식당/카페_pct","domain_식음료_pct",
                       "domain_운동/스포츠_pct","domain_운세_pct","domain_의료/건강_pct","domain_채용_pct","domain_커머스_pct"),

    use_clr=True,
    extra_col_weights=None,

    # 안정화 옵션
    prior_mix=0.2,
    prior_from="mda_mean",   # "mda_mean" | "uniform" | "none"
    prior_bg_dict=None,
    use_idf=False,
    idf_smooth=1.0,
    min_similarity=None
):
    # share 피처
    share_cols = [c for c in mda_pf.columns if c.startswith('share_cat')]
    if not share_cols:
        raise ValueError("mda_pf에 share_cat* 컬럼이 없습니다. 먼저 enrichment를 수행하세요.")

    # 존재하는 컬럼만 사용
    def _keep_exist(cols): return [c for c in cols if c in mda_pf.columns]
    volume_cols         = _keep_exist(volume_cols)
    size_ratio_cols     = _keep_exist(size_ratio_cols)
    os_ratio_cols       = _keep_exist(os_ratio_cols)
    category_ratio_cols = _keep_exist(category_ratio_cols)
    domain_ratio_cols   = _keep_exist(domain_ratio_cols)

    # prior 배경 분포
    prior_bg = None
    if prior_bg_dict is not None:
        prior_bg = dict(prior_bg_dict)
    elif prior_from == "mda_mean":
        avg = mda_pf[share_cols].fillna(0.0).mean(axis=0)
        s = avg.sum()
        if s > 0:
            prior_bg = (avg / s).to_dict()
    elif prior_from == "uniform":
        prior_bg = {c: 1.0/len(share_cols) for c in share_cols}

    # 가중치(광고 분포) 생성
    col_w = make_ad_pair_weights_from_ad_df(
        ad_df, power=weight_power, min_frac=min_pair_frac,
        prior_mix=prior_mix if prior_mix else 0.0,
        prior_bg=prior_bg
    )
    if top_weight_feats:
        top_keys = set(pd.Series(col_w).sort_values(ascending=False).head(top_weight_feats).index)
        col_w = {k: (v if k in top_keys else 0.0) for k,v in col_w.items()}

    # IDF 보정(옵션)
    if use_idf:
        df_share = (mda_pf[share_cols].fillna(0) != 0).sum(axis=0)
        N = len(mda_pf)
        idf = np.log((N + 1.0) / (df_share + idf_smooth))
        idf = idf / (idf.mean() + 1e-12)
        for k in list(col_w.keys()):
            if k in idf.index:
                col_w[k] *= float(idf[k])

    if extra_col_weights:
        col_w.update(extra_col_weights)

    # 피처 행렬
    X, all_feat_cols = build_feature_matrix_plus(
        mda_pf,
        share_cols=share_cols,
        volume_cols=volume_cols,
        size_ratio_cols=size_ratio_cols,
        os_ratio_cols=os_ratio_cols,
        category_ratio_cols=category_ratio_cols,
        domain_ratio_cols=domain_ratio_cols,
        use_clr=use_clr,
        col_weights=col_w,
        zscore=True
    )

    # 앵커/센트로이드
    used = set(ad_df['mda_idx'].astype(int))
    anchors = (ad_df.sort_values(top_anchor_by, ascending=False)
                   .drop_duplicates('mda_idx')
                   .head(n_anchor)['mda_idx']
                   .astype(int).tolist())
    anchors = [m for m in anchors if m in X.index]
    if not anchors:
        raise ValueError("anchor가 없습니다. ad_df에 상위 매체가 있는지 확인하세요.")
    centroid = X.loc[anchors].mean(axis=0).values

    # 후보 & 필터
    cand = mda_pf[~mda_pf['mda_idx'].isin(used)].copy()
    if 'basic_classification' in cand.columns and exclude_classes:
        cand = cand[~cand['basic_classification'].isin(exclude_classes)]
    if 'days_active' in cand.columns:
        cand = cand[cand['days_active'] >= min_days_active]
    if cand.empty:
        return pd.DataFrame(columns=['mda_idx','similarity']), anchors, all_feat_cols, col_w

    # 유사도
    B = X.loc[cand['mda_idx']].values
    cand['similarity'] = cosine_vec(centroid, B)
    if (min_similarity is not None):
        cand = cand[cand['similarity'] >= float(min_similarity)]
        if cand.empty:
            return pd.DataFrame(columns=['mda_idx','similarity']), anchors, all_feat_cols, col_w

    # 예측 블렌딩(옵션)
    has_pred = (blend_pred_table is not None) and (blend_ad_id is not None)
    if has_pred:
        pt = blend_pred_table[blend_pred_table['ads_idx']==blend_ad_id][['mda_idx','pred_turn']].copy()
        cand = cand.merge(pt, on='mda_idx', how='left')
        cand['pred_turn'] = cand['pred_turn'].fillna(0.0)
        maxv = cand['pred_turn'].max()
        cand['pred_norm'] = cand['pred_turn'] / (maxv + 1e-9)
        cand['final_score'] = blend_alpha*cand['similarity'] + (1.0-blend_alpha)*cand['pred_norm']

    # 정렬
    if sort_by == "pred" and has_pred:
        sort_key = "pred_turn"
    elif sort_by == "sim":
        sort_key = "similarity"
    else:
        sort_key = "final_score" if has_pred else "similarity"

    keep = [c for c in [
        'mda_idx','similarity','final_score','pred_turn','pred_norm',
        'basic_classification','days_active','conversion_rate',
        'expected_total_profit','total_ads'
    ] if c in cand.columns]
    out = cand[keep].sort_values(sort_key, ascending=False).head(topN).reset_index(drop=True)
    return out, anchors, all_feat_cols, col_w
# === /통합 셀 끝 ===


In [ ]:
out, anchors, feats, w = recommend_with_weighted_similarity(
    ad_df=ads_9935_pf,
    mda_pf=mda_pf_enriched,
    use_clr=True,          # 비율 전부 CLR
    weight_power=0.5,      # 루트 가중
    prior_mix=0.2,         # 배경 분포 섞기
    prior_from="mda_mean", # mda 평균 분포
    n_anchor=5
)
display(out.head(20))

## 신규 광고 유사도, 전환율예측

In [ ]:
# ==============================================================
# 신규(가상) 광고 → 유사 광고 코호트 기반 매체별 CVR/전환수 예측 (원샷 셀)
# ==============================================================

import numpy as np
import pandas as pd

# ----------------- 파일 경로 -----------------
PERF_CSV = "/Users/Jiwon/Documents/GitHub/final_project/Jiwon/수정_시간별적립보고서(최종).csv"   # 시간별 집계(또는 로그)
META_CSV = "/Users/Jiwon/Documents/GitHub/final_project/Jiwon/광고도메인리스트.csv"             # 기존 광고 메타(ads_idx 존재)
NEW_ADS_CSV = "/Users/Jiwon/Documents/GitHub/final_project/Jiwon/신규가상광고.csv"                                                    # 신규 가상광고 목록

# ----------------- 하이퍼파라미터 -----------------
L_DAYS = 30                 # 예측에 사용할 과거 창 길이
H_DAYS = 30                 # 시나리오B(향후 H일) 클릭/전환 예측 길이
K = 10                      # 유사 광고 코호트 크기
BETA_SIM = 1.0              # 유사도 가중 지수 (cos sim^beta)
ALPHA_PRIOR = 2.0           # 베타-바이노믹 스무딩 alpha
BETA_PRIOR = 120.0          # 베타-바이노믹 스무딩 beta
BLEND_KAPPA = 15.0          # 블렌딩 전환: eff/(eff+kappa)
DOMAIN_WEIGHT = 1.0         # 도메인 가중치(영향 키우려면 2~3)
RESTRICT_SAME_DOMAIN = False# True면 같은 도메인 후보만 코호트로
DROP_RARE_MIN_ADS = 3       # 희귀 원-핫 열 제거(3개 미만 광고에서만 등장)

CAT_COLS = ["domain", "ads_category", "ads_os_type", "ads_type", "ads_rejoin_type"]
PRICE_CANDIDATES = ("ads_media_price", "media_price", "contract_price")

# ----------------- 작은 유틸 -----------------
def _norm_meta(df):
    df = df.copy()
    for c in CAT_COLS:
        if c in df.columns:
            df[c] = df[c].astype(str).str.strip()
    return df

def _pick_date_col(df):
    for c in ["rpt_time_date","click_day","click_date"]:
        if c in df.columns:
            return c
    raise ValueError("날짜 컬럼(rpt_time_date / click_day / click_date) 없음")

def _clicks_convs_cols(df):
    clicks = None; convs = None
    if "rpt_time_clk" in df.columns: clicks = "rpt_time_clk"
    elif "clicks" in df.columns:     clicks = "clicks"
    elif "click_key" in df.columns:  clicks = None           # 로그면 size()로 산출

    if "rpt_time_turn" in df.columns: convs = "rpt_time_turn"
    elif "conversions" in df.columns: convs = "conversions"
    elif "conversion" in df.columns:  convs = "conversion"
    return clicks, convs

def _z(s):
    s = pd.to_numeric(s, errors="coerce")
    mu, sd = np.nanmean(s), np.nanstd(s)
    sd = 1.0 if (sd is None or sd == 0 or np.isnan(sd)) else sd
    return (s - mu) / sd, float(mu), float(sd)

# ----------------- 1) 기존 광고로 피처 공간 학습 -----------------
def build_feature_space(ad_meta, drop_rare_min_ads=3, domain_weight=1.0):
    meta = _norm_meta(ad_meta).drop_duplicates("ads_idx").copy()

    X_list = []
    group_cols = {}

    # (a) 범주형 원-핫
    for c in CAT_COLS:
        if c in meta.columns:
            one = pd.get_dummies(meta[c].astype(str), prefix=c, dtype=float)
            if c == "domain" and domain_weight != 1.0:
                one = one * float(domain_weight)
            X_list.append(one)
            group_cols[c] = list(one.columns)

    # (b) 수치형(가격) - 후보 중 존재하는 첫 컬럼 사용
    price_col = next((c for c in PRICE_CANDIDATES if c in meta.columns), None)
    price_mu = price_sd = None
    if price_col is not None:
        price_log = np.log1p(pd.to_numeric(meta[price_col], errors="coerce"))
        price_z, price_mu, price_sd = _z(price_log)
        X_list.append(price_z.to_frame("price_z"))

    if not X_list:
        raise ValueError("ad_meta에서 만들 수 있는 피처가 없습니다.")

    X = pd.concat(X_list, axis=1).fillna(0.0)
    # 희귀 원-핫 열 제거
    if drop_rare_min_ads and drop_rare_min_ads > 1:
        nz = (X != 0).sum(0)
        keep = nz[nz >= float(drop_rare_min_ads)].index
        X = X[keep]
        # 그룹 열 목록 갱신
        for g in list(group_cols.keys()):
            group_cols[g] = [col for col in group_cols[g] if col in X.columns]

    # z-score (열 단위)
    mu = X.mean()
    sd = X.std(ddof=0).replace(0, 1.0)
    A_z = (X - mu) / (sd + 1e-9)
    A_z.index = meta["ads_idx"].astype(int).values

    store = dict(
        A_z=A_z.astype(np.float32),
        mu=mu.astype(np.float32), sd=sd.astype(np.float32),
        cols=A_z.columns.tolist(),
        group_cols=group_cols,
        price_col=price_col,
        price_mu=price_mu, price_sd=price_sd,
        meta_small=meta[["ads_idx"] + [c for c in CAT_COLS if c in meta.columns]]
    )
    return store

# ----------------- 2) 신규 광고 1건 인코딩(기존 공간에 맞춤) -----------------
def encode_new_ad_row(row, store):
    cols = store["cols"]
    x = pd.Series(0.0, index=cols, dtype=float)

    # 범주형: 기존에 있던 열만 1로 세움(새로운 카테고리는 정보 없음 → 0)
    for c in CAT_COLS:
        if c in row.index:
            val = str(row[c]).strip()
            one_col = f"{c}_{val}"
            if one_col in x.index:
                x[one_col] = 1.0

    # 가격: log1p 후 기존 z스케일 사용
    pcol = store["price_col"]
    if pcol and pcol in row.index:
        val = pd.to_numeric(row[pcol], errors="coerce")
        if pd.notnull(val):
            z = (np.log1p(val) - store["price_mu"]) / (store["price_sd"] + 1e-9)
            if "price_z" in x.index:
                x["price_z"] = float(z)

    # L2 정규화용 벡터 반환
    a = x.values.astype(np.float32)
    a = a / (np.linalg.norm(a) + 1e-12)
    return x, a

# ----------------- 3) 신규 광고 → 유사 광고 코호트 추출 -----------------
def cohort_for_new_ad(row, store, K=50, beta=BETA_SIM, restrict_same_domain=False):
    A = store["A_z"]
    # 후보 제한(같은 도메인)
    cand = A
    if restrict_same_domain and "domain" in row.index and "domain" in store["meta_small"].columns:
        dom = str(row["domain"]).strip()
        ok_ids = store["meta_small"].loc[
            store["meta_small"]["domain"].astype(str).str.strip() == dom, "ads_idx"
        ].astype(int)
        cand = A.loc[A.index.intersection(ok_ids)]

    # 인코딩
    x, a = encode_new_ad_row(row, store)
    M = cand.values
    norms = np.sqrt((M*M).sum(1)) + 1e-12
    sims = (M @ a) / norms

    if len(sims) == 0:
        return pd.DataFrame(columns=["weight","sim"])

    k = min(K, len(sims))
    top = np.argpartition(-sims, k-1)[:k]
    top = top[np.argsort(-sims[top])]

    sim_vals = sims[top]
    w = np.power(np.clip(sim_vals, 0, 1), beta); w = w / (w.sum() + 1e-12)

    out = pd.DataFrame({"ads_idx": cand.index.values[top].astype(int),
                        "sim": sim_vals, "weight": w})
    out = out.set_index("ads_idx")
    return out

# ----------------- 4) 코호트 기반 매체사 CVR/전환수 예측 -----------------
def predict_media_from_cohort(perf_df, ad_meta_df, cohort_df, new_row,
                              L_days=30, H_days=30,
                              alpha_prior=ALPHA_PRIOR, beta_prior=BETA_PRIOR,
                              blend_kappa=BLEND_KAPPA):
    if cohort_df.empty:
        return pd.DataFrame(), {"window_end": None, "L_days": L_days, "H_days": H_days}

    perf = perf_df.copy()
    date_col = _pick_date_col(perf)
    perf[date_col] = pd.to_datetime(perf[date_col])
    wend = perf[date_col].max().normalize()
    start = wend - pd.Timedelta(days=L_days-1)
    hist = perf[(perf[date_col]>=start) & (perf[date_col]<=wend)].copy()

    clk_col, cv_col = _clicks_convs_cols(hist)

    # ads_category를 히스토리에 붙임(매체×카테 베이스라인용)
    if "ads_category" in ad_meta_df.columns and "ads_category" not in hist.columns:
        cat_map = ad_meta_df.drop_duplicates("ads_idx").set_index("ads_idx")["ads_category"]
        hist = hist.merge(cat_map.rename("ads_category"), left_on="ads_idx", right_index=True, how="left")

    # 코호트 가중 집계
    sub = hist[hist["ads_idx"].isin(cohort_df.index)].copy()
    if sub.empty:
        return pd.DataFrame(), {"window_end": str(wend.date()), "L_days": L_days, "H_days": H_days}

    if clk_col is None:
        g = sub.groupby(["ads_idx","mda_idx"]).agg(
            clicks=("ads_idx","size"), convs=("conversion","sum")
        ).reset_index()
    else:
        g = sub.groupby(["ads_idx","mda_idx"]).agg(
            clicks=(clk_col,"sum"), convs=(cv_col,"sum")
        ).reset_index()

    w_map = cohort_df["weight"].to_dict()
    g["w"] = g["ads_idx"].map(w_map).fillna(0.0)
    g["w_clicks"] = g["w"] * g["clicks"]
    g["w_convs"]  = g["w"] * g["convs"]

    agg = g.groupby("mda_idx").agg(
        cohort_eff_clicks=("w_clicks","sum"),
        cohort_eff_convs=("w_convs","sum"),
        coverage_ads=("ads_idx","nunique")
    )

    # 베이스라인 (매체 전체)
    if clk_col is None:
        base_m = hist.groupby("mda_idx").agg(
            clicks=("ads_idx","size"), convs=("conversion","sum")
        )
    else:
        base_m = hist.groupby("mda_idx").agg(
            clicks=(clk_col,"sum"), convs=(cv_col,"sum")
        )
    base_m["cvr_m"] = (base_m["convs"] + alpha_prior) / (base_m["clicks"] + alpha_prior + beta_prior)

    # 베이스라인 (매체×카테고리: 신규 광고의 카테고리 사용)
    tcat = None
    if "ads_category" in new_row.index:
        try:
            tcat = int(pd.to_numeric(new_row["ads_category"], errors="coerce"))
        except Exception:
            tcat = None

    base_mc = pd.DataFrame()
    if (tcat is not None) and ("ads_category" in hist.columns):
        subcat = hist[hist["ads_category"]==tcat]
        if not subcat.empty:
            if clk_col is None:
                base_mc = subcat.groupby("mda_idx").agg(
                    clicks=("ads_idx","size"), convs=("conversion","sum")
                )
            else:
                base_mc = subcat.groupby("mda_idx").agg(
                    clicks=(clk_col,"sum"), convs=(cv_col,"sum")
                )
            base_mc["cvr_mc"] = (base_mc["convs"] + alpha_prior) / (base_mc["clicks"] + alpha_prior + beta_prior)

    out = agg.join(base_m[["cvr_m"]], how="left").join(base_mc[["cvr_mc"]], how="left").fillna({"cvr_m":0.0})
    out["cvr_cohort"] = (out["cohort_eff_convs"] + alpha_prior) / (out["cohort_eff_clicks"] + alpha_prior + beta_prior)
    base = out["cvr_mc"].fillna(out["cvr_m"])
    eff = out["cohort_eff_clicks"]
    w1 = eff / (eff + float(blend_kappa))
    out["pred_cvr"] = w1 * out["cvr_cohort"] + (1.0 - w1) * base
    out["per_1000_clicks_conv"] = out["pred_cvr"] * 1000.0

    # 시나리오: 코호트 일평균 클릭 × H_days
    if clk_col is None:
        per_day = (sub.groupby(["mda_idx", sub[date_col].dt.normalize()])["ads_idx"]
                   .size().rename("clk").reset_index())
    else:
        per_day = (sub.groupby(["mda_idx", sub[date_col].dt.normalize()])[clk_col]
                   .sum().rename("clk").reset_index())
    daily = per_day.groupby("mda_idx")["clk"].mean()
    out["scenarioB_clicks"] = daily.reindex(out.index).fillna(0.0).values * float(H_DAYS)
    out["scenarioB_conv"]   = out["pred_cvr"] * out["scenarioB_clicks"]

    out = out.reset_index().sort_values("per_1000_clicks_conv", ascending=False).reset_index(drop=True)
    info = {"window_end": str(wend.date()), "L_days": L_DAYS, "H_days": H_DAYS}
    return out, info

# ----------------- 5) 배치 실행: 신규 광고 목록 전체 처리 -----------------
def run_new_ads_batch(new_ads_df, ad_meta_df, perf_df,
                      topN_media=20, show_first_n=1):
    store = build_feature_space(ad_meta_df, drop_rare_min_ads=DROP_RARE_MIN_ADS,
                                domain_weight=DOMAIN_WEIGHT)

    results = {}  # key: new_ad_key → (pred_df, cohort_df, info)
    # 신규 광고의 key 컬럼 결정(ads_idx가 없으면 행번호 사용)
    key_col = "ads_idx" if "ads_idx" in new_ads_df.columns else None

    for i, row in new_ads_df.iterrows():
        new_key = int(row[key_col]) if key_col else int(i)
        cohort = cohort_for_new_ad(row, store, K=K, beta=BETA_SIM,
                                   restrict_same_domain=RESTRICT_SAME_DOMAIN)
        pred, info = predict_media_from_cohort(perf_df, ad_meta_df, cohort, row,
                                               L_days=L_DAYS, H_days=H_DAYS,
                                               alpha_prior=ALPHA_PRIOR, beta_prior=BETA_PRIOR,
                                               blend_kappa=BLEND_KAPPA)
        results[new_key] = (pred, cohort, info)

    # 미리보기
    shown = 0
    for k, (pred, cohort, info) in results.items():
        print(f"\n=== 신규광고 {k} : window_end={info['window_end']}, L_days={info['L_days']} ===")
        if not cohort.empty:
            disp = cohort.reset_index().rename(columns={"index":"ads_idx"})[["ads_idx","sim","weight"]].head(10)
            display(disp.style.format({"sim":"{:.3f}","weight":"{:.3f}"}))
        else:
            print("코호트가 비어 있습니다.")

        if not pred.empty:
            cols = ["mda_idx","pred_cvr","per_1000_clicks_conv",
                    "cohort_eff_clicks","coverage_ads",
                    "cvr_m","cvr_mc","cvr_cohort",
                    "scenarioB_clicks","scenarioB_conv"]
            display(pred.head(topN_media)[[c for c in cols if c in pred.columns]]
                    .style.format({"pred_cvr":"{:.6f}","per_1000_clicks_conv":"{:.3f}",
                                   "cvr_m":"{:.6f}","cvr_mc":"{:.6f}","cvr_cohort":"{:.6f}",
                                   "scenarioB_clicks":"{:.3f}","scenarioB_conv":"{:.3f}"}))
        else:
            print("히스토리 구간에서 코호트 데이터가 없습니다.")
        shown += 1
        if shown >= show_first_n:
            break

    return results

# ================== 실행 ==================
# 1) 데이터 로드(네 파일 스키마에 맞춰 자동 인식)
# perf_df = pd.read_csv(PERF_CSV, encoding="utf-8-sig")
# ad_meta_df = pd.read_csv(META_CSV, encoding="utf-8-sig")
# new_ads_df = pd.read_csv(NEW_ADS_CSV, encoding="utf-8-sig")

# # 2) 배치 실행 (상위 1개 신규광고만 미리보기)
# results = run_new_ads_batch(new_ads_df, ad_meta_df, perf_df,
#                             topN_media=20, show_first_n=1)

# 3) 특정 신규광고 결과 꺼내쓰기 예:
# new_id = list(results.keys())[0]
# pred_df, cohort_df, info = results[new_id]
# display(pred_df.head(30))
# display(cohort_df.head(20))
# print(info)


# 신규광고 유사도

In [ ]:
# ==============================================================
# 신규(가상) 광고 → 유사 광고 코호트 기반 매체별 CVR/전환수 예측 (원샷 셀)
# ==============================================================

import numpy as np
import pandas as pd

# ----------------- 파일 경로 -----------------
PERF_CSV = "/Users/Jiwon/Documents/GitHub/final_project/Jiwon/수정_시간별적립보고서(최종).csv"   # 시간별 집계(또는 로그)
META_CSV = "/Users/Jiwon/Documents/GitHub/final_project/Jiwon/광고도메인리스트.csv"             # 기존 광고 메타(ads_idx 존재)
NEW_ADS_CSV = "/Users/Jiwon/Documents/GitHub/final_project/Jiwon/신규가상광고.csv"                                                    # 신규 가상광고 목록

# ----------------- 하이퍼파라미터 -----------------
L_DAYS = 30                 # 예측에 사용할 과거 창 길이
H_DAYS = 30                 # 시나리오B(향후 H일) 클릭/전환 예측 길이
K = 10                      # 유사 광고 코호트 크기
BETA_SIM = 1.0              # 유사도 가중 지수 (cos sim^beta)
ALPHA_PRIOR = 2.0           # 베타-바이노믹 스무딩 alpha
BETA_PRIOR = 120.0          # 베타-바이노믹 스무딩 beta
BLEND_KAPPA = 15.0          # 블렌딩 전환: eff/(eff+kappa)
DOMAIN_WEIGHT = 1.0         # 도메인 가중치(영향 키우려면 2~3)
RESTRICT_SAME_DOMAIN = False# True면 같은 도메인 후보만 코호트로
DROP_RARE_MIN_ADS = 3       # 희귀 원-핫 열 제거(3개 미만 광고에서만 등장)

CAT_COLS = ["domain", "ads_category", "ads_os_type", "ads_type", "ads_rejoin_type"]
PRICE_CANDIDATES = ("ads_media_price", "media_price", "contract_price")

# ----------------- 작은 유틸 -----------------
def _norm_meta(df):
    df = df.copy()
    for c in CAT_COLS:
        if c in df.columns:
            df[c] = df[c].astype(str).str.strip()
    return df

def _pick_date_col(df):
    for c in ["rpt_time_date","click_day","click_date"]:
        if c in df.columns:
            return c
    raise ValueError("날짜 컬럼(rpt_time_date / click_day / click_date) 없음")

def _clicks_convs_cols(df):
    clicks = None; convs = None
    if "rpt_time_clk" in df.columns: clicks = "rpt_time_clk"
    elif "clicks" in df.columns:     clicks = "clicks"
    elif "click_key" in df.columns:  clicks = None           # 로그면 size()로 산출

    if "rpt_time_turn" in df.columns: convs = "rpt_time_turn"
    elif "conversions" in df.columns: convs = "conversions"
    elif "conversion" in df.columns:  convs = "conversion"
    return clicks, convs

def _z(s):
    s = pd.to_numeric(s, errors="coerce")
    mu, sd = np.nanmean(s), np.nanstd(s)
    sd = 1.0 if (sd is None or sd == 0 or np.isnan(sd)) else sd
    return (s - mu) / sd, float(mu), float(sd)

# ----------------- 1) 기존 광고로 피처 공간 학습 -----------------
def build_feature_space(ad_meta, drop_rare_min_ads=3, domain_weight=1.0):
    meta = _norm_meta(ad_meta).drop_duplicates("ads_idx").copy()

    X_list = []
    group_cols = {}

    # (a) 범주형 원-핫
    for c in CAT_COLS:
        if c in meta.columns:
            one = pd.get_dummies(meta[c].astype(str), prefix=c, dtype=float)
            if c == "domain" and domain_weight != 1.0:
                one = one * float(domain_weight)
            X_list.append(one)
            group_cols[c] = list(one.columns)

    # (b) 수치형(가격) - 후보 중 존재하는 첫 컬럼 사용
    price_col = next((c for c in PRICE_CANDIDATES if c in meta.columns), None)
    price_mu = price_sd = None
    if price_col is not None:
        price_log = np.log1p(pd.to_numeric(meta[price_col], errors="coerce"))
        price_z, price_mu, price_sd = _z(price_log)
        X_list.append(price_z.to_frame("price_z"))

    if not X_list:
        raise ValueError("ad_meta에서 만들 수 있는 피처가 없습니다.")

    X = pd.concat(X_list, axis=1).fillna(0.0)
    # 희귀 원-핫 열 제거
    if drop_rare_min_ads and drop_rare_min_ads > 1:
        nz = (X != 0).sum(0)
        keep = nz[nz >= float(drop_rare_min_ads)].index
        X = X[keep]
        # 그룹 열 목록 갱신
        for g in list(group_cols.keys()):
            group_cols[g] = [col for col in group_cols[g] if col in X.columns]

    # z-score (열 단위)
    mu = X.mean()
    sd = X.std(ddof=0).replace(0, 1.0)
    A_z = (X - mu) / (sd + 1e-9)
    A_z.index = meta["ads_idx"].astype(int).values

    store = dict(
        A_z=A_z.astype(np.float32),
        mu=mu.astype(np.float32), sd=sd.astype(np.float32),
        cols=A_z.columns.tolist(),
        group_cols=group_cols,
        price_col=price_col,
        price_mu=price_mu, price_sd=price_sd,
        meta_small=meta[["ads_idx"] + [c for c in CAT_COLS if c in meta.columns]]
    )
    return store

# ----------------- 2) 신규 광고 1건 인코딩(기존 공간에 맞춤) -----------------
def encode_new_ad_row(row, store):
    cols = store["cols"]
    x = pd.Series(0.0, index=cols, dtype=float)

    # 범주형: 기존에 있던 열만 1로 세움(새로운 카테고리는 정보 없음 → 0)
    for c in CAT_COLS:
        if c in row.index:
            val = str(row[c]).strip()
            one_col = f"{c}_{val}"
            if one_col in x.index:
                x[one_col] = 1.0

    # 가격: log1p 후 기존 z스케일 사용
    pcol = store["price_col"]
    if pcol and pcol in row.index:
        val = pd.to_numeric(row[pcol], errors="coerce")
        if pd.notnull(val):
            z = (np.log1p(val) - store["price_mu"]) / (store["price_sd"] + 1e-9)
            if "price_z" in x.index:
                x["price_z"] = float(z)

    # L2 정규화용 벡터 반환
    a = x.values.astype(np.float32)
    a = a / (np.linalg.norm(a) + 1e-12)
    return x, a

# ----------------- 3) 신규 광고 → 유사 광고 코호트 추출 -----------------
def cohort_for_new_ad(row, store, K=50, beta=BETA_SIM, restrict_same_domain=False):
    A = store["A_z"]
    # 후보 제한(같은 도메인)
    cand = A
    if restrict_same_domain and "domain" in row.index and "domain" in store["meta_small"].columns:
        dom = str(row["domain"]).strip()
        ok_ids = store["meta_small"].loc[
            store["meta_small"]["domain"].astype(str).str.strip() == dom, "ads_idx"
        ].astype(int)
        cand = A.loc[A.index.intersection(ok_ids)]

    # 인코딩
    x, a = encode_new_ad_row(row, store)
    M = cand.values
    norms = np.sqrt((M*M).sum(1)) + 1e-12
    sims = (M @ a) / norms

    if len(sims) == 0:
        return pd.DataFrame(columns=["weight","sim"])

    k = min(K, len(sims))
    top = np.argpartition(-sims, k-1)[:k]
    top = top[np.argsort(-sims[top])]

    sim_vals = sims[top]
    w = np.power(np.clip(sim_vals, 0, 1), beta); w = w / (w.sum() + 1e-12)

    out = pd.DataFrame({"ads_idx": cand.index.values[top].astype(int),
                        "sim": sim_vals, "weight": w})
    out = out.set_index("ads_idx")
    return out

# ----------------- 4) 코호트 기반 매체사 CVR/전환수 예측 -----------------
def predict_media_from_cohort(perf_df, ad_meta_df, cohort_df, new_row,
                              L_days=30, H_days=30,
                              alpha_prior=ALPHA_PRIOR, beta_prior=BETA_PRIOR,
                              blend_kappa=BLEND_KAPPA):
    if cohort_df.empty:
        return pd.DataFrame(), {"window_end": None, "L_days": L_days, "H_days": H_days}

    perf = perf_df.copy()
    date_col = _pick_date_col(perf)
    perf[date_col] = pd.to_datetime(perf[date_col])
    wend = perf[date_col].max().normalize()
    start = wend - pd.Timedelta(days=L_days-1)
    hist = perf[(perf[date_col]>=start) & (perf[date_col]<=wend)].copy()

    clk_col, cv_col = _clicks_convs_cols(hist)

    # ads_category를 히스토리에 붙임(매체×카테 베이스라인용)
    if "ads_category" in ad_meta_df.columns and "ads_category" not in hist.columns:
        cat_map = ad_meta_df.drop_duplicates("ads_idx").set_index("ads_idx")["ads_category"]
        hist = hist.merge(cat_map.rename("ads_category"), left_on="ads_idx", right_index=True, how="left")

    # 코호트 가중 집계
    sub = hist[hist["ads_idx"].isin(cohort_df.index)].copy()
    if sub.empty:
        return pd.DataFrame(), {"window_end": str(wend.date()), "L_days": L_days, "H_days": H_days}

    if clk_col is None:
        g = sub.groupby(["ads_idx","mda_idx"]).agg(
            clicks=("ads_idx","size"), convs=("conversion","sum")
        ).reset_index()
    else:
        g = sub.groupby(["ads_idx","mda_idx"]).agg(
            clicks=(clk_col,"sum"), convs=(cv_col,"sum")
        ).reset_index()

    w_map = cohort_df["weight"].to_dict()
    g["w"] = g["ads_idx"].map(w_map).fillna(0.0)
    g["w_clicks"] = g["w"] * g["clicks"]
    g["w_convs"]  = g["w"] * g["convs"]

    agg = g.groupby("mda_idx").agg(
        cohort_eff_clicks=("w_clicks","sum"),
        cohort_eff_convs=("w_convs","sum"),
        coverage_ads=("ads_idx","nunique")
    )

    # 베이스라인 (매체 전체)
    if clk_col is None:
        base_m = hist.groupby("mda_idx").agg(
            clicks=("ads_idx","size"), convs=("conversion","sum")
        )
    else:
        base_m = hist.groupby("mda_idx").agg(
            clicks=(clk_col,"sum"), convs=(cv_col,"sum")
        )
    base_m["cvr_m"] = (base_m["convs"] + alpha_prior) / (base_m["clicks"] + alpha_prior + beta_prior)

    # 베이스라인 (매체×카테고리: 신규 광고의 카테고리 사용)
    tcat = None
    if "ads_category" in new_row.index:
        try:
            tcat = int(pd.to_numeric(new_row["ads_category"], errors="coerce"))
        except Exception:
            tcat = None

    base_mc = pd.DataFrame()
    if (tcat is not None) and ("ads_category" in hist.columns):
        subcat = hist[hist["ads_category"]==tcat]
        if not subcat.empty:
            if clk_col is None:
                base_mc = subcat.groupby("mda_idx").agg(
                    clicks=("ads_idx","size"), convs=("conversion","sum")
                )
            else:
                base_mc = subcat.groupby("mda_idx").agg(
                    clicks=(clk_col,"sum"), convs=(cv_col,"sum")
                )
            base_mc["cvr_mc"] = (base_mc["convs"] + alpha_prior) / (base_mc["clicks"] + alpha_prior + beta_prior)

    out = agg.join(base_m[["cvr_m"]], how="left").join(base_mc[["cvr_mc"]], how="left").fillna({"cvr_m":0.0})
    out["cvr_cohort"] = (out["cohort_eff_convs"] + alpha_prior) / (out["cohort_eff_clicks"] + alpha_prior + beta_prior)
    base = out["cvr_mc"].fillna(out["cvr_m"])
    eff = out["cohort_eff_clicks"]
    w1 = eff / (eff + float(blend_kappa))
    out["pred_cvr"] = w1 * out["cvr_cohort"] + (1.0 - w1) * base
    out["per_1000_clicks_conv"] = out["pred_cvr"] * 1000.0

    # 시나리오: 코호트 일평균 클릭 × H_days
    if clk_col is None:
        per_day = (sub.groupby(["mda_idx", sub[date_col].dt.normalize()])["ads_idx"]
                   .size().rename("clk").reset_index())
    else:
        per_day = (sub.groupby(["mda_idx", sub[date_col].dt.normalize()])[clk_col]
                   .sum().rename("clk").reset_index())
    daily = per_day.groupby("mda_idx")["clk"].mean()
    out["scenarioB_clicks"] = daily.reindex(out.index).fillna(0.0).values * float(H_DAYS)
    out["scenarioB_conv"]   = out["pred_cvr"] * out["scenarioB_clicks"]

    out = out.reset_index().sort_values("per_1000_clicks_conv", ascending=False).reset_index(drop=True)
    info = {"window_end": str(wend.date()), "L_days": L_DAYS, "H_days": H_DAYS}
    return out, info

# ----------------- 5) 배치 실행: 신규 광고 목록 전체 처리 -----------------
def run_new_ads_batch(new_ads_df, ad_meta_df, perf_df,
                      topN_media=20, show_first_n=1):
    store = build_feature_space(ad_meta_df, drop_rare_min_ads=DROP_RARE_MIN_ADS,
                                domain_weight=DOMAIN_WEIGHT)

    results = {}  # key: new_ad_key → (pred_df, cohort_df, info)
    key_col = "ads_idx" if "ads_idx" in new_ads_df.columns else None

    for i, row in new_ads_df.iterrows():
        new_key = int(row[key_col]) if key_col else int(i)
        cohort = cohort_for_new_ad(row, store, K=K, beta=BETA_SIM,
                                   restrict_same_domain=RESTRICT_SAME_DOMAIN)
        pred, info = predict_media_from_cohort(perf_df, ad_meta_df, cohort, row,
                                               L_days=L_DAYS, H_days=H_DAYS,
                                               alpha_prior=ALPHA_PRIOR, beta_prior=BETA_PRIOR,
                                               blend_kappa=BLEND_KAPPA)
        results[new_key] = (pred, cohort, info)

    # 미리보기: show_first_n > 0 인 경우에만 출력
    if show_first_n and show_first_n > 0:
        shown = 0
        for k, (pred, cohort, info) in results.items():
            print(f"\n=== 신규광고 {k} : window_end={info['window_end']}, L_days={info['L_days']} ===")

            if cohort is not None and not cohort.empty:
                disp = cohort.reset_index()[["ads_idx","sim","weight"]].head(10)
                try:
                    display(disp.style.format({"sim":"{:.3f}","weight":"{:.3f}"}))
                except Exception:
                    print(disp.to_string(index=False))
            else:
                print("코호트가 비어 있습니다.")

            if pred is not None and not pred.empty:
                cols = ["mda_idx","pred_cvr","per_1000_clicks_conv",
                        "cohort_eff_clicks","coverage_ads",
                        "cvr_m","cvr_mc","cvr_cohort",
                        "scenarioB_clicks","scenarioB_conv"]
                view = pred.head(topN_media)[[c for c in cols if c in pred.columns]]
                try:
                    display(view.style.format({
                        "pred_cvr":"{:.6f}", "per_1000_clicks_conv":"{:.3f}",
                        "cvr_m":"{:.6f}", "cvr_mc":"{:.6f}", "cvr_cohort":"{:.6f}",
                        "scenarioB_clicks":"{:.3f}", "scenarioB_conv":"{:.3f}"
                    }))
                except Exception:
                    print(view.to_string(index=False))
            else:
                print("히스토리 구간에서 코호트 데이터가 없습니다.")

            shown += 1
            if shown >= show_first_n:
                break

    return results





In [ ]:
# ================== 실행 (단일 ads_idx만 미리보기) ==================
# 1) 데이터 로드
perf_df = pd.read_csv(PERF_CSV, encoding="utf-8-sig")
ad_meta_df = pd.read_csv(META_CSV, encoding="utf-8-sig")
new_ads_df = pd.read_csv(NEW_ADS_CSV, encoding="utf-8-sig")

# 2) sanity check
assert "ads_idx" in new_ads_df.columns, "new_ads_df에 'ads_idx' 컬럼이 없습니다."

# 3) 모든 신규광고에 대해 계산만 수행(미리보기 출력 없음)
results = run_new_ads_batch(new_ads_df, ad_meta_df, perf_df,
                            topN_media=20, show_first_n=0)

target_ads_idx = 500005  

if target_ads_idx not in results:
    print(f"[!] ads_idx {target_ads_idx} 가 results에 없습니다. new_ads_df에 있는지 확인하세요.")
else:
    pred, cohort, info = results[target_ads_idx]

# 코호트 상위 10개
if cohort is not None and not cohort.empty:
    disp = cohort.reset_index()[["ads_idx","sim","weight"]].head(10)
    try:
        display(disp.style.format({"sim":"{:.3f}","weight":"{:.3f}"}))
    except Exception:
        print(disp.to_string(index=False))
else:
    print("코호트가 비어 있습니다.")

# 매체사 추천 테이블(topN_media)
if pred is not None and not pred.empty:
    cols = ["mda_idx","pred_cvr","per_1000_clicks_conv",
            "cohort_eff_clicks","coverage_ads",
            "cvr_m","cvr_mc","cvr_cohort",
            "scenarioB_clicks","scenarioB_conv"]
    view = pred.head(20)[[c for c in cols if c in pred.columns]]
    try:
        display(view.style.format({
            "pred_cvr":"{:.6f}", "per_1000_clicks_conv":"{:.3f}",
            "cvr_m":"{:.6f}", "cvr_mc":"{:.6f}", "cvr_cohort":"{:.6f}",
            "scenarioB_clicks":"{:.3f}", "scenarioB_conv":"{:.3f}"
        }))
    except Exception:
        print(view.to_string(index=False))
else:
    print("히스토리 구간에서 코호트 데이터가 없습니다.")
